# **Launch Sites Locations Analysis with Folium**


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis notebook, we have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this notebook, we will be performing more interactive visual analytics using `Folium`.


## Objectives


This notebook contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, we can able to find some geographical patterns about launch sites.


Let's first import required Python packages for this notebook:


In [1]:
import folium
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [3]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df=pd.read_csv(URL)

In [4]:
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, we can take a look at what are the coordinates for each site.


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give us any intuitive insights about where are those launch sites. If we are very good at geography, we can interpret those numbers directly in our mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and we can find a small yellow circle near the city of Houston and we can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


In [8]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for lat,lon,lnc in zip(launch_sites_df['Lat'],launch_sites_df['Long'],launch_sites_df['Launch Site']):
    circle=folium.Circle([lat,lon], radius=1000, color='#d35400', fill=True).add_child(folium.Popup(lnc))
    site_map.add_child(circle)

    marker = folium.map.Marker(
    [lat,lon],
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % lnc,
        )
    )
    site_map.add_child(marker)

In [9]:
site_map

Now, we can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Solution:

* Launch sites such as 'KSC LC-39A','CCAFS SLC-40' and 'CCAFS LC-40' are closer to the Equator line than the launch site 'VAFB SLC-4E'.  

* From the above map it is observed that all the launch sites are in very close proximity to the coast.


## Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.


In [10]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [11]:
marker_cluster = MarkerCluster()


In [12]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
    

In [13]:
marker_color=[]
for i in spacex_df['class']:
    if i==1:
        marker_color.append('green')
    else:
        marker_color.append('red')

spacex_df['marker_color']=marker_color

In [14]:
spacex_df.head()

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


In [15]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed
for lat,lon,clr in zip(spacex_df['Lat'],spacex_df['Long'],spacex_df['marker_color']):
    circle=folium.Circle([lat,lon], radius=1000, color=clr, fill=True)
    marker_cluster.add_child(circle)

site_map

From the color-labeled markers in marker clusters, we can able to easily identify which launch sites have relatively high success rates.


## TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while we are exploring the map, we can easily find the coordinates of any points of interests (such as railway)


In [16]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if we can easily find any railway, highway, coastline, etc. Move mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [17]:
from math import sin, cos, sqrt, atan2, radians

In [18]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

### Coastline distance

In [19]:
# find coordinate of the closet coastline
launch_site_lat=28.57331
launch_site_lon=-80.64692
coastline_lat=28.58024
coastline_lon=-80.64245
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [20]:
distance=distance_coastline

In [21]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
    icon_size=(20,20),
    icon_anchor=(0,0),
    html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
    )
)
site_map.add_child(distance_marker)

In [22]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
lines=folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]], weight=10)
site_map.add_child(lines)

Similarly, we can draw a line betwee a launch site to its closest city, railway, highway, etc. We need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [23]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


### City distance

In [24]:
launch_site_lat_city=28.57331
launch_site_lon_city=-80.64692
city_lat=28.07461
city_lon=-80.6115
distance_city = calculate_distance(launch_site_lat_city, launch_site_lon_city, city_lat, city_lon)

In [25]:
distance_marker = folium.Marker(
    [city_lat, city_lon],
    icon=DivIcon(
    icon_size=(20,20),
    icon_anchor=(0,0),
    html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
    )
)
site_map.add_child(distance_marker)

In [26]:
lines=folium.PolyLine(locations=[[launch_site_lat_city, launch_site_lon_city], [city_lat, city_lon]], weight=10)
site_map.add_child(lines)

### Railway distance

In [27]:
launch_site_lat_rail=28.57331
launch_site_lon_rail=-80.64692
rail_lat=28.5804
rail_lon=-80.59087
distance_rail = calculate_distance(launch_site_lat_rail, launch_site_lon_rail, rail_lat, rail_lon)

In [28]:
distance_marker = folium.Marker(
    [rail_lat, rail_lon],
    icon=DivIcon(
    icon_size=(20,20),
    icon_anchor=(0,0),
    html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_rail),
    )
)
site_map.add_child(distance_marker)

In [29]:
lines=folium.PolyLine(locations=[[launch_site_lat_rail, launch_site_lon_rail], [rail_lat, rail_lon]], weight=10)
site_map.add_child(lines)

### Highway distance

In [30]:
launch_site_lat_high=28.57331
launch_site_lon_high=-80.64692
high_lat=28.57289
high_lon=-80.65551
distance_high = calculate_distance(launch_site_lat_high, launch_site_lon_high, high_lat, high_lon)

In [31]:
distance_marker = folium.Marker(
    [high_lat, high_lon],
    icon=DivIcon(
    icon_size=(20,20),
    icon_anchor=(0,0),
    html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_high),
    )
)
site_map.add_child(distance_marker)

In [32]:
lines=folium.PolyLine(locations=[[launch_site_lat_high, launch_site_lon_high], [high_lat, high_lon]], weight=10)
site_map.add_child(lines)

In [33]:
site_map

After we plot distance lines to the proximities, we can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?



Solution:

* Launch sites are in close proximity to the railways. For example, the launch site 'KSC LC-39A' is at 5.53 KM from the 'NASA Railroad'. By having launch sites close to railways, logistics become more manageable, ensuring timely delivery of critical resources.

* Launch sites are in close proximity to the highways. For example, the launch site 'KSC LC-39A' is at 0.84 KM from the 'Kennedy Parkway North'. The proximity to highways plays a vital role in ensuring smooth operations during pre-launch preparations and post-launch activities.

* Launch sites are in close proximity to the coastline. For example, the launch site 'KSC LC-39A' is at 0.89 KM from the nearest coastline. The proximity to the coastline is chosen so that if something goes wrong during the rocket’s ascent, debris will fall into the ocean rather than densely populated areas. Launching over water minimizes the risk to human lives and property. 

* Launch sites keep certain distance away from the cities. For example, 'Melbourne' which is one of the nearest city of the launch site 'KSC LC-39A' is at a distance of 55.58 KM. Launch sites are typically located far away from densely populated areas. This minimizes the risk to human life and property in case of accidents during launch or booster separation.